In [1]:
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import execute_values
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
import json
import pprint

from RAG.insertion_agent import InsertionAgent

In [5]:
rag_insertion_agent = InsertionAgent(elderly_id="87654321-4321-4321-4321-019876543210")

INFO:root:Loading embedding model: google/embeddinggemma-300m
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: google/embeddinggemma-300m
INFO:sentence_transformers.SentenceTransformer:14 prompts are loaded, with the keys: ['query', 'document', 'BitextMining', 'Clustering', 'Classification', 'InstructionRetrieval', 'MultilabelClassification', 'PairClassification', 'Reranking', 'Retrieval', 'Retrieval-query', 'Retrieval-document', 'STS', 'Summarization']
INFO:root:embedding model loaded successfully: google/embeddinggemma-300m


#### Insert Test User Profile

In [13]:
with open("../test_cases/knowledge_base.json") as f:
    knowledge_base = json.load(f)

test_user_profile = knowledge_base["test_elderly_profile"]
#pprint.pprint(test_user_profile)
HC_records = knowledge_base["HCM_data"]
#pprint.pprint(HC_records)
LTM_records = knowledge_base["LTM_data"]
pprint.pprint(LTM_records)

[{'category': 'Family', 'key': 'Daughter-in-law', 'value': 'Meena Singh'},
 {'category': 'Family', 'key': 'Ganesh Menon Dhoby', 'value': 'eldest son'},
 {'category': 'lifestyle', 'key': 'friend', 'value': 'Mr Lim'},
 {'category': 'lifestyle',
  'key': 'routine',
  'value': 'Morning walks with Mr Lim at 7.30am every day'},
 {'category': 'family',
  'key': 'family_relationship',
  'value': 'Children and grandchildren usually join birthday celebrations'},
 {'category': 'lifestyle', 'key': 'likes', 'value': 'jazz music'},
 {'category': 'lifestyle', 'key': 'hobby', 'value': 'gardening'},
 {'category': 'lifestyle',
  'key': 'routine',
  'value': 'soft boiled eggs and coffee everyday for breakfast'},
 {'category': 'lifestyle',
  'routine_name': 'morning routine',
  'value': 'morning walk at 7.30am'},
 {'category': 'lifestyle',
  'key': 'hobby',
  'value': 'used to play badminton in school days'}]


In [7]:
rag_insertion_agent.insert_elderly_profile(test_user_profile)

#### Insert Test Medical Records

In [10]:
for hc in HC_records:
    rag_insertion_agent.insert_health_record(
        record_type=hc["type"],
        description=hc["description"],
        diagnosis_date=hc.get("date", None)
    )

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:root:❌ Database error inserting health record: (psycopg2.errors.InvalidTextRepresentation) invalid input value for enum record_type_enum: "allergy"
LINE 5: ...          '87654321-4321-4321-4321-019876543210', 'allergy',...
                                                             ^

[SQL: 
                    INSERT INTO healthcare_records (
                        elderly_id, record_type, description, diagnosis_date, embedding
                    ) VALUES (
                        %(elderly_id)s, %(record_type)s, %(description)s, %(diagnosis_date)s, %(embedding)s
                    )
                    RETURNING id, last_updated;
                ]
[parameters: {'elderly_id': '87654321-4321-4321-4321-019876543210', 'record_type': 'allergy', 'description': 'Patient is allergic to penicillin which causes rash.', 'diagnosis_date': None, 'embedding': '[-0.13455796241760254, -0.0011422340758144855, 0.022401699796319008, -0.0012216466711834073, -0.005779414437711239, 0.00733632547

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Insert LTM for Test Patient

In [15]:
for ltm in LTM_records:
    rag_insertion_agent.insert_long_term(
        category=ltm["category"],
        key=ltm["key"],
        value=ltm["value"]
    )


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:root:❌ Database error inserting LTM: (psycopg2.errors.InvalidTextRepresentation) invalid input value for enum ltm_category_enum: "Family"
LINE 5: ...          '87654321-4321-4321-4321-019876543210', 'Family', ...
                                                             ^

[SQL: 
                    INSERT INTO long_term_memory (
                        elderly_id, category, key, value, embedding
                    ) VALUES (
                        %(elderly_id)s, %(category)s, %(key)s, %(value)s, %(embedding)s
                    )
                    RETURNING id, last_updated;
                ]
[parameters: {'elderly_id': '87654321-4321-4321-4321-019876543210', 'category': 'Family', 'key': 'Daughter-in-law', 'value': 'Meena Singh', 'embedding': '[-0.19209453463554382, -0.0069421264342963696, 0.030603842809796333, 0.0010050470009446144, 0.015940669924020767, 0.028256410732865334, -0.0308287180 ... (16763 characters truncated) ... 6687417626381, 0.020578838884830475, -0.013

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:root:❌ Database error inserting LTM: (psycopg2.errors.InvalidTextRepresentation) invalid input value for enum ltm_category_enum: "Family"
LINE 5: ...          '87654321-4321-4321-4321-019876543210', 'Family', ...
                                                             ^

[SQL: 
                    INSERT INTO long_term_memory (
                        elderly_id, category, key, value, embedding
                    ) VALUES (
                        %(elderly_id)s, %(category)s, %(key)s, %(value)s, %(embedding)s
                    )
                    RETURNING id, last_updated;
                ]
[parameters: {'elderly_id': '87654321-4321-4321-4321-019876543210', 'category': 'Family', 'key': 'Ganesh Menon Dhoby', 'value': 'eldest son', 'embedding': '[-0.1904672235250473, 0.010585030540823936, 0.020697293803095818, 0.006491549778729677, -0.003271430963650346, 0.03341001272201538, -0.03062237985432 ... (16721 characters truncated) ... 348258018494, 0.010101082734763622, -0.03

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyError: 'key'

#### Insert Short Term Memory for Test Patient

#### Clear Test Patient Data


In [ ]:
# -----------------------------------------------------------------------
# 1. Connect to Neon Postgres
# -----------------------------------------------------------------------
load_dotenv()
conn_string = os.getenv("DATABASE_URL")
conn = None

try:
    with psycopg2.connect(conn_string) as conn:
        print("Connection established")
        with conn.cursor() as cur:

# -----------------------------------------------------------------------
# 2. Delete Test Patient Data
# -----------------------------------------------------------------------
            elderly_id = '87654321-4321-4321-4321-019876543210'
            tables = ['healthcare_records', 'long_term_memory', 'short_term_memory', 'elderly_profiles']
            for table in tables:
                cur.execute(f"DELETE FROM {table} WHERE elderly_id = %s", (elderly_id,))
                print(f"Deleted records from {table} for elderly_id {elderly_id}")

            # Commit the changes
            conn.commit()
            print("All test patient data cleared successfully")

except Exception as e:
    print(f"Error clearing test patient data: {e}")
    if conn:
        conn.rollback()
finally:
    if conn:
        conn.close()
        print("Connection closed")
